In [73]:
# !pip install datasets
# !pip install librosa

In [97]:
import IPython.display as pds
import numpy as np
import pandas as pd
import librosa
import warnings
import random
import zipfile
import os
import re
import json
import tensorflow as tf
warnings.filterwarnings("ignore")

from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [75]:
!git clone https://github.com/Bangkit-Capstone-Team/IOH-Chat-App.git

fatal: destination path 'IOH-Chat-App' already exists and is not an empty directory.


In [76]:
SPEECH_DATA_DIR = "/content/IOH-Chat-App/Machine Learning/datasets/speech/audio.zip"

zf = zipfile.ZipFile(SPEECH_DATA_DIR)
zf.extractall("/tmp")
zf.close()

In [98]:
DATASET_PATH = "/tmp/audio/"
JSON_PATH = "/tmp/speech.json"

In [144]:
one_sec_signal = 22050

def prepare_dataset(dataset_path, json_path, **kwargs):
  json_data = {
      "mapping": list(),
      "features": list(),
      "labels": list(),
  }

  for i, (dirpath, _, files) in enumerate(os.walk(dataset_path)):
    for audio in sorted(files):
      
      filename = os.path.join(dirpath, audio)
      string_label = re.findall(r"^\w*",audio)[0]

      if os.path.getsize(filename) != 0:
        signal, sr = librosa.load(filename, sr=16000)

        if len(signal) >= one_sec_signal:
          signal = signal[:one_sec_signal]
          mfcc = librosa.feature.mfcc(signal, **kwargs)
        
          json_data["mapping"].append(string_label)
          json_data["features"].append(mfcc)
          json_data["labels"].append(i)

          i += 1

  return json_data

In [ ]:
dataset = prepare_dataset(DATASET_PATH, JSON_PATH, n_mfcc=13, n_fft=2048, hop_length=512)

In [ ]:
df = pd.DataFrame(dataset)
df.tail()

In [ ]:
features = df.features.values
labels = df.labels.values

x_train, y_train, x_test, y_test = train_test_split(features, labels, test_size=0.2, random_state=1)

x_train = np.expand_dims(x_train, axis=1)
x_test = np.expand_dims(x_test, axis=1)

input_shape = (x_train.shape[0], x_train.shape[1], 1)
print(input_shape)

In [134]:
def build_model(n_class, input_shape):
  model = tf.keras.Sequential()

  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
  model.add(layers.MaxPooling2D(2, 2))

  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(2, 2))
  
  model.add(layers.Flatten())

  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(n_class, activation='softmax'))

  return model

In [135]:
LR = 1e-4
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LR)
LOSS = tf.keras.losses.sparse_categorical_crossentropy

model = build_model(len(y_train), input_shape)

model.compile(optimizer=OPTIMIZER,
              loss=LOSS,
              metrics=["accuracy"])

model.summary()

ValueError: ignored

In [ ]:
EPOCHS = 15

model.fit(x_train, 
          y_train, 
          epochs=EPOCHS, 
          validation_data=(x_test, y_test), 
          batch_size=128)